## Nomenclatura dos Dados
* _p: Previous
    * _p _win_pct: Acertos prévios

In [292]:
import requests 
import json
import pandas as pd
pd.set_option('display.max_columns', 200)
import numpy as np
import itertools
import pickle
import os

In [293]:
def pickle_data(path, data):
    if not path.endswith('.pickle'):
        path += '.pickle'
    with open(path, 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return

In [294]:
def read_pickle(path):
    with open(path, 'rb') as handle:
        data = pickle.load(handle)
    return data

In [295]:
def fetch_data(week, year=2019):
    url = "https://api.nflpickwatch.com/v1/nfl/expert-picks-su/" + str(year) + "/" + str(week)
    resp = requests.get(url).text
    data = json.loads(resp)
    return data

In [296]:
def get_weekly_picks(week, data):
    games = pd.json_normalize(data['games'])[['id', 'rt_id', 'ht_id', 'game_state']]
    games.index = games.id
    cols = ['user_id']
    for g in list(games.id):
        cols.append("picks." + str(g) + ".t")
    picks = pd.json_normalize(data['expertPicks'])[cols].T
    picks.columns = picks.loc['user_id']
    picks = picks.drop(['user_id'])
    picks.index = games.index
    
    gp = pd.concat([games, picks], axis=1)
    gp = gp.fillna(0)

    gp = gp.rename(columns={"game_state":"game_result"})
    gp['game_result'] = np.where(gp['game_result']=='home-win', 1, 
                                np.where(gp['game_result']=='road-win', -1, 0))

    teams_dict = {}
    for ht in gp.ht_id:
        teams_dict[ht] = 1
    for rt in gp.rt_id:
        teams_dict[rt] = -1
    gp.iloc[:,4:] = gp.iloc[:,4:].replace(teams_dict)

    return gp

In [394]:
tg = pd.DataFrame(columns=['id', 'rt_id', 'ht_id', 'game_result', 'vote', 'abs_vote', 'vote_diff'])

In [424]:
def get_votes(week, rank, picks, top, strategy="rank_squared", tight=tg):
    best = rank.sort_values(by='week' + str(week-1) + '_rank')
    best = best.iloc[0:top]
#     best['votes'] = (top - best['week' + str(week-1) + '_rank'])
    
    if strategy == "rank_squared":
        best['pct_rank'] = best['week' + str(week-1) + '_rank'].rank(pct=True, ascending=False)
        best['votes'] = best.pct_rank**2
        best['votes'] = np.where(best['votes'] < 0, 0, best['votes'])
        
    elif strategy == 'democracy':
        best['votes'] = 1
    
    elif strategy == 'n-rank':
        best['votes'] = best['votes'] = (top - best['week' + str(week-1) + '_rank'] + top/2)/(top/2)
        
    
    bid = list(best.user_id)
    
    votes = picks[bid]*best['votes']
    votes['id'] = picks.id
    votes['rt_id'] = picks['rt_id']
    votes['ht_id'] = picks['ht_id']
    votes['game_result'] = picks['game_result']
    
    votes['abs_vote'] = best['votes'].sum()
    votes['vote'] = votes[bid].sum(axis=1)
    
    votes['vote_ratio'] = votes.vote.abs()/votes.abs_vote
    
    temp = votes[['id', 'rt_id','ht_id','game_result','vote','abs_vote', 'vote_ratio']]
    temp = temp[temp.vote_ratio <= 0.1]
    temp['vote_first'] = best.iloc[0].votes

    tight = pd.concat([tight, temp])
        
    
    votes['vote'] = np.where(votes['vote']==0, votes[best.iloc[0].user_id], votes['vote']) #In case of votes summing up to zero, the first ranked expert gets the Minerva vote.
    
    votes['bet'] = np.sign(votes.vote)
    return votes, tight

In [306]:
year = 2019
bla = pd.read_csv('Data/' + str(year) + '/rank_' + str(year) +'.csv')
bla.index = bla.user_id

In [386]:
top = 25
week = 15
bla = calculate_rank(bla, season_hits_modifier=0, hist_modifier=5, hist_decay=True, last_season_bonus=0.3)
best = bla.sort_values(by='week' + str(week-1) + '_rank')
best = best.iloc[0:top]
# best['pct_rank'] = best['week' + str(week-1) + '_rank'].rank(pct=True, ascending=False)
best['votes'] = (top - best['week' + str(week-1) + '_rank'] + top/2)/(top/2)
# best['votes'] = np.where(best['votes'] == 0, 0, best['votes'])
best

bid = list(best.user_id)
picks = season[week]
votes = picks[bid]*best['votes']
votes['abs_vote'] = best['votes'].sum()
votes['vote'] = votes.sum(axis=1)
votes['vote_ratio'] = votes.vote/votes.abs_vote 

votes

,76,166,181,156,31,189,96,198,46077,185,51,172,157,26895,264,265,27765,161,88,262,37,75,41282,235,160,abs_vote,vote,vote_ratio
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1551,2.92,2.84,2.76,2.68,2.52,2.52,2.44,2.36,2.28,2.12,2.12,1.96,1.96,1.88,1.72,1.72,1.64,1.4,1.4,1.4,0.00,1.08,1.08,1.08,0.92,48.12,94.92,1.972569
1560,-2.92,-2.84,2.76,-2.68,-2.52,2.52,-2.44,-2.36,-2.28,-2.12,-2.12,-1.96,-1.96,-1.88,1.72,-1.72,-1.64,-1.4,-1.4,-1.4,-1.32,-1.08,-1.08,-1.08,-0.92,48.12,14.00,0.290939
1553,2.92,2.84,-2.76,2.68,2.52,2.52,-2.44,2.36,2.28,2.12,2.12,1.96,1.96,1.88,1.72,1.72,-1.64,1.4,1.4,1.4,1.32,1.08,1.08,1.08,0.92,48.12,82.56,1.715711
1554,2.92,2.84,2.76,2.68,2.52,2.52,2.44,2.36,2.28,2.12,2.12,1.96,1.96,1.88,1.72,1.72,-1.64,1.4,1.4,1.4,1.32,1.08,1.08,1.08,0.92,48.12,92.96,1.931837
1555,2.92,2.84,2.76,2.68,2.52,2.52,2.44,2.36,2.28,2.12,2.12,1.96,1.96,1.88,1.72,1.72,1.64,1.4,1.4,1.4,1.32,-1.08,1.08,1.08,0.92,48.12,94.08,1.955112
1556,2.92,2.84,-2.76,-2.68,2.52,2.52,2.44,2.36,-2.28,2.12,2.12,1.96,1.96,1.88,-1.72,-1.72,-1.64,1.4,-1.4,1.4,1.32,1.08,1.08,1.08,0.92,48.12,67.84,1.409809
1557,-2.92,-2.84,-2.76,-2.68,-2.52,-2.52,-2.44,-2.36,-2.28,-2.12,-2.12,-1.96,-1.96,-1.88,-1.72,-1.72,-1.64,-1.4,-1.4,-1.4,-1.32,-1.08,-1.08,-1.08,-0.92,48.12,0.00,0.000000
1558,-2.92,-2.84,2.76,-2.68,-2.52,-2.52,2.44,-2.36,-2.28,-2.12,-2.12,-1.96,-1.96,-1.88,-1.72,-1.72,-1.64,-1.4,-1.4,-1.4,-1.32,-1.08,1.08,-1.08,-0.92,48.12,12.56,0.261014
1559,-2.92,-2.84,-2.76,-2.68,-2.52,-2.52,-2.44,-2.36,-2.28,-2.12,-2.12,-1.96,-1.96,-1.88,-1.72,-1.72,-1.64,-1.4,-1.4,-1.4,-1.32,-1.08,-1.08,-1.08,-0.92,48.12,0.00,0.000000


In [378]:
del picks

In [345]:
votes

NameError: name 'votes' is not defined

In [ ]:
tg = np.where(votes.vote.sum().abs()/best.votes.sum() <= best.iloc[0].votes, )

In [383]:
season = read_pickle('Data/2019/picks_2019.pickle')

In [342]:
picks[1]

,id,rt_id,ht_id,game_result,51,106,27064,76,98,172,110,66,111,156,161,205,65,168,121,160,17,262,27,69,214,285,282,130,70,264,29,31,112,146,235,61,78,189,90,103,23,185,27773,114,190,157,26365,75,107,32,261,266,88,181,26895,265,188,271,67,34,213,186,283,237,101,174,284,217,280,260,96,35,49,234,198,199,191,27768,270,238,22,259,251,95,80,27765,48,37,256,27769,240,41288,41282,57,37563,45189,220,38321,40108,38323,40890,47801,41287,41759,37564,46077,41755,176,45128,41285,53447,44212,166,40506,44216
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1336,1336,GB,CHI,-1,1,-1,-1,1,1,1,1,1,1,1,1,1,1,-1,1,1,1,1,-1,-1,1,-1,-1,1,1,-1,1,1,1,1,1,1,-1,1,1,1,-1,1,1,1,1,1,-1,-1,1,1,1,1,1,1,-1,-1,1,-1,1,-1,1,1,-1,1,-1,-1,1,1,-1,1,1,-1,-1,-1,1,1,-1,0,1,0,-1,-1,-1,1,-1,1,1,0,-1,1,0,1,1,-1,1,-1,-1,1,1,-1,1,-1,-1,1,-1,1,-1,-1,0,1,0,0,-1,-1,-1
1344,1344,ATL,MIN,1,1,-1,1,1,1,1,1,1,1,1,1,1,1,-1,1,1,1,1,-1,-1,1,1,-1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,-1,1,1,1,1,1,-1,-1,1,1,1,1,-1,1,-1,1,-1,1,1,1,-1,-1,1,1,1,1,-1,1,1,1,1,-1,1,0,1,-1,1,1,-1,1,1,1,1,1,0,-1,1,1,1,-1,-1,1,-1,1,1,1,-1,1,-1,1,1,1,1,1,0,0,1,1,-1
1338,1338,BAL,MIA,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,-1,-1,1,-1,-1,-1,-1,-1,-1,1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,-1,-1,-1
1339,1339,BUF,NYJ,-1,-1,-1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,1,1,1,1,1,1,-1,1,1,1,1,1,1,1,1,1,-1,1,1,-1,1,1,0,-1,1,1,-1,1,1,1,1,1,1,-1,-1,-1,-1,1,1,-1,1,-1,1,1,-1,1,1,1,-1,1,-1,1,1,1,1,1,1,1,-1,0,-1,1,-1,1,1,-1,1,1,-1,1,0,1,1,-1,-1,-1,1,-1,1,1,1,1,-1,1,1,1,1,1,-1,1,0,0,-1,-1,1
1347,1347,KC,JAX,-1,1,1,-1,-1,-1,-1,-1,1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,0,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,1,-1,-1,-1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,0,-1,-1,-1
1341,1341,LAR,CAR,-1,1,1,1,-1,1,1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,1,1,-1,-1,1,1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,0,1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,1,1,-1,1,1,-1,-1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,1,-1,1,-1,-1,1,-1,-1,-1,1,0,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,1,-1,0,0,-1,-1,-1
1337,1337,TEN,CLE,-1,1,1,1,1,1,1,1,1,1,1,1,1,-1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,1,1,-1,1,1,-1,-1,1,1,1,-1,1,1,1,-1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,-1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,-1,1
1340,1340,WAS,PHI,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1
1351,1351,CIN,SEA,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,0,1,0,0,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1


In [303]:
x=int(top/2)
print (best.votes.iloc[0]/best.votes.sum())
print (best.votes/best.votes.sum())

0.053435114503816786
user_id
280      0.053435
75       0.052345
265      0.050164
26895    0.050164
121      0.049073
285      0.047983
264      0.046892
78       0.044711
146      0.044711
103      0.042530
90       0.042530
35       0.041439
76       0.040349
110      0.039258
205      0.037077
111      0.037077
160      0.035987
270      0.034896
70       0.033806
112      0.030534
31       0.030534
189      0.030534
23       0.028353
185      0.028353
27064    0.027263
Name: votes, dtype: float64


In [307]:
def get_rank(year):
    data = fetch_data(year=year, week=1)
    rank = pd.json_normalize(data['expertPicks'])[['user_id', 'name', 'affiliation', 'prev_rank', 'p_wins', 'p_win_pct']]
    rank['season_hits'] = 0
    rank['week0_rank'] = rank.prev_rank.rank(method='max')
    ht = ['week' + str(w) + '_hits' for w in range (1,18)]
    rt = ['week' + str(w) + '_rate' for w in range (1,18)]
    rk = ['week' + str(w) + '_rank' for w in range (1,18)]

    for w in range (0,17):
        rank[ht[w]] = 0
        rank[rt[w]] = 0
        rank[rk[w]] = 0

    rank.index = rank.user_id
    
    for week in range (1, 18):
        data = fetch_data(week, year)
        week_pct = pd.json_normalize(data['expertPicks'])[['user_id', 't_wins']]
        week_pct.index = week_pct.user_id
        rank['week'+str(week)+"_hits"]=week_pct.t_wins
        for d in data['expertPicks']:
            if d['picks'] == {} and d['user_id'] in rank.user_id:
                rank = rank.drop(d['user_id'])
                print("Removed " + str(d['user_id']))
        

    return rank

In [308]:
def calculate_rank(rank, hist_modifier, season_hits_modifier, hist_decay, last_season_bonus):
    for week in range(1, 18):
        rate = 'week' + str(week) + "_rate"
        rank[rate] = 0
        rank['season_hits'] = 0
        for w in range (1, week+1):
            rank['season_hits'] += rank['week'+str(w)+'_hits']

        else:
            if hist_decay:
                hd = {}
                width = max(0,(week-hist_modifier))
                rg = min(hist_modifier, week)
                for i in range(rg):
                    hd[week-rg+i+1] = (i+1)/(week-width)
                    
            n = week
            while (n > week - hist_modifier) and (n > 0):
                if hist_decay:
                    rank[rate] += rank['week' + str(n) + "_hits"]*hd[n]
#                     print("Week: {}; n: {}; Decay: {}".format(week, n, hd[n]))
            
                else:
                    rank[rate] += rank['week' + str(n) + "_hits"]   
                n -= 1
            
            bonus_weeks = 6
            rank[rate] += rank['p_wins']/16 * max(bonus_weeks-week,0) * last_season_bonus
            rank[rate] = rank[rate]/(min(hist_modifier, week))

        rank[rate] += (rank['season_hits'] * season_hits_modifier)/week
        rank['week' + str(week) + "_rank"] = rank[rate].rank(method='max', ascending=False)
    
    return rank 

In [ ]:
rank = pd.read_csv('Data/2019/rank_2019.csv')

In [ ]:
week = 4
bonus_weeks = 4
last_season_bonus = 0.2
(180/16) * max(bonus_weeks-week+1, 0) * last_season_bonus/bonus_weeks

In [420]:
def win_a_bet(hp):
    tg = pd.DataFrame(columns=['id', 'rt_id', 'ht_id', 'game_result', 'vote', 'abs_vote', 'vote_ratio', 'vote_first'])
    results_cols = []
    hits_cols=[]
    gms_cols = []
    for y in range(2014,2020):
        results_cols+= [str(y)+"_hits", str(y)+"_gms", str(y)+"_rate"]
        hits_cols.append(str(y)+'_hits')
        gms_cols.append(str(y)+'_gms')

    results = pd.DataFrame(columns=results_cols)

    for year in range (2014,2020):
        season = read_pickle('Data/' + str(year) + '/' + "picks_" + str(year) + ".pickle")
        total_hits = 0
        total_games = 0
        for week in range(1,18):
            rank = pd.read_csv('Data/' + str(year) + '/rank_' + str(year) +'.csv')
            rank.index = rank.user_id
            rank = calculate_rank(rank, hist_modifier=hp['hist_modifier'], season_hits_modifier=hp['season_hits_modifier'], hist_decay=hp['hist_decay'], last_season_bonus=hp['last_season_bonus'])

    #         season[week] = get_weekly_picks(week, data)
            picks = season[week]

            votes, tg = get_votes(week, rank, picks, top=hp['n_top_experts'], strategy=hp['vote_strategy'], tight=tg)
            picks['vote'] = votes.vote
            picks['bet'] = votes.bet
            picks['hit'] = np.where(picks.bet == picks.game_result, 1, 0)

            hits = picks['hit'].sum()
            games = picks['hit'].count()
            total_hits += hits
            total_games += games

            results.loc['week'+str(week), str(year)+'_gms'] = games
            results.loc['week'+str(week), str(year)+'_hits'] = hits
            results.loc['week'+str(week), str(year)+'_rate'] = hits/games

        results.loc['total', str(year)+'_gms'] = total_games
        results.loc['total', str(year)+'_hits'] = total_hits
        results.loc['total', str(year)+'_rate'] = total_hits/total_games
        print (tg.shape)

        print("{} Hits - Total: {}/{} = {:.2%}".format(year,total_hits,total_games,total_hits/total_games))

    results['mean_hits'] = results[hits_cols].mean(1)
    results['total_hits'] = results[hits_cols].sum(1)
    results['mean_gms'] = results[gms_cols].mean(1)
    results['total_gms'] = results[gms_cols].sum(1)
    results['total_rate'] = results['total_hits']/results['total_gms']
    #         print("Hits - Week {}: {}/{} = {:.2%}".format(week,hits,games,hits/games))

    return results, tg

In [ ]:
season = {}
total_hits = 0
total_games = 0
for week in range(1,18):
    data = fetch_data(week)
    season[week] = get_weekly_picks(week, data)
    picks = season[week]
    votes = get_votes(week, rank, picks, top=_TOP_N)
    picks['vote'] = votes.vote
    picks['bet'] = votes.bet
    picks['hit'] = np.where(picks.bet == picks.game_result, 1, 0)
    hits = picks['hit'].sum()
    games = picks['hit'].count()
    total_hits += hits
    total_games += games
    print("Hits - Week {}: {}/{} = {:.2%}".format(week,hits,games,hits/games))
print("Hits - TOTAL: {}/{} = {:.2%}".format(total_hits,total_games,total_hits/total_games))

In [ ]:
rank=get_rank()

In [310]:
def gather_data(year):
    season = {}
    for week in range(1,18):
        data = fetch_data(week, year)
        season[week] = get_weekly_picks(week, data)
        
    if not os.path.exists('Data/' + str(year)):
        os.makedirs('Data/' + str(year))
        
    pickle_data(path='Data/' + str(year) + '/picks_' + str(year) + '.pickle', data=season)     
    
    return

In [23]:
for year in range(2014,2016):
    r = get_rank(year)
    r = calculate_rank(r, hist_modifier=hyperparameters['hist_modifier'], season_hits_modifier=hyperparameters['season_hits_modifier'], consider_last_season=hyperparameters['consider_last_season'])
    r.to_csv('Data/' + str(year) + '/rank_' + str(year) +'.csv', index=False)

Removed 127
Removed 124
Removed 123
Removed 143
Removed 36
Removed 128
Removed 113
Removed 30
Removed 71
Removed 102
Removed 122
Removed 153
Removed 170
Removed 169
Removed 58
Removed 135
Removed 167
Removed 122
Removed 156
Removed 54
Removed 69


In [ ]:
%%time
for year in range(2014, 2016):
    print(year)
    gather_data(year)

In [349]:
hyperparameters = {
    'hist_modifier': 4,
    'n_top_experts': 18,
    'last_season_bonus': 0.5,
    'season_hits_modifier': 0,
    'vote_strategy': 'n-rank',
    'hist_decay': True,
}

In [425]:
r, t = win_a_bet(hyperparameters)

(26, 8)
2014 Hits - Total: 172/256 = 67.19%
(37, 8)
2015 Hits - Total: 158/256 = 61.72%
(47, 8)
2016 Hits - Total: 163/256 = 63.67%
(56, 8)
2017 Hits - Total: 171/256 = 66.80%
(69, 8)
2018 Hits - Total: 166/256 = 64.84%
(82, 8)
2019 Hits - Total: 162/256 = 63.28%


In [426]:
t

,id,rt_id,ht_id,game_result,vote,abs_vote,vote_ratio,vote_first
1,1,GB,SEA,1,-144.000000,-216.000000,-0.666667,-12.000000
2,2,NO,ATL,1,168.000000,-216.000000,-0.777778,-12.000000
3,3,MIN,LAR,-1,0.000000,-216.000000,-0.000000,-12.000000
4,4,CLE,PIT,1,-216.000000,-216.000000,-1.000000,-12.000000
5,5,JAX,PHI,1,-216.000000,-216.000000,-1.000000,-12.000000
...,...,...,...,...,...,...,...,...
1516,1516,GB,SF,1,1.444444,32.111111,0.044983,2.888889
1543,1543,SF,NO,-1,2.111111,33.444444,0.063123,2.888889
1545,1545,LAC,JAX,-1,-2.111111,33.444444,0.063123,2.888889
1546,1546,KC,NE,-1,-2.111111,33.444444,0.063123,2.888889


In [430]:
t['bet']=np.sign(t.vote)
t['hits']=np.where(t.bet==t.game_result, 1, 0)
t[t.game_result==1].count()

id             45
rt_id          45
ht_id          45
game_result    45
vote           45
abs_vote       45
vote_ratio     45
vote_first     45
bet            45
hits           45
dtype: int64

In [364]:
tg

,id,rt_id,ht_id,game_result,vote,abs_vote


In [320]:
ntop_list = []
for hm in range(1,13):
    hyperparameters['hist_modifier']=hm
    ntop_change = {}
    for nt in range (5,26):
        print ((" HM = " + str(hm) + "; nTop = " +str(nt) + " ").center(80,'='))
        hyperparameters['n_top_experts'] = nt
        hyperparameters['season_hits_modifier'] = hm/4
        results = win_a_bet(hyperparameters)
        print("Grand-Total: {}/{} = {:.2%}".format(results.loc['total','total_hits'],results.loc['total','total_gms'],results.loc['total','total_rate']))
        print(''.center(80,'=') + '\n')
        ntop_change[nt]=results.loc['total','total_rate']
    ntop_list.append(ntop_change)

=============================== HM = 1; nTop = 5 ===============================
2014 Hits - Total: 166/256 = 64.84%
2015 Hits - Total: 158/256 = 61.72%
2016 Hits - Total: 162/256 = 63.28%
2017 Hits - Total: 167/256 = 65.23%
2018 Hits - Total: 161/256 = 62.89%
2019 Hits - Total: 165/256 = 64.45%
Grand-Total: 979.0/1536.0 = 63.74%

=============================== HM = 1; nTop = 6 ===============================
2014 Hits - Total: 169/256 = 66.02%
2015 Hits - Total: 162/256 = 63.28%
2016 Hits - Total: 163/256 = 63.67%
2017 Hits - Total: 171/256 = 66.80%
2018 Hits - Total: 162/256 = 63.28%
2019 Hits - Total: 162/256 = 63.28%
Grand-Total: 989.0/1536.0 = 64.39%

=============================== HM = 1; nTop = 7 ===============================
2014 Hits - Total: 173/256 = 67.58%
2015 Hits - Total: 159/256 = 62.11%
2016 Hits - Total: 167/256 = 65.23%
2017 Hits - Total: 166/256 = 64.84%
2018 Hits - Total: 162/256 = 63.28%
2019 Hits - Total: 165/256 = 64.45%
Grand-Total: 992.0/1536.0 = 64.58%

=

In [321]:
ntop = pd.DataFrame(ntop_list).T

cols = ["hm"+str(c+1) for c in list(ntop.columns)]

ntop.columns = cols

for c in list(ntop.columns):
    ntop['rank' + (c[2:])] = ntop[c].rank(pct=True)

In [326]:
ntop.iloc[:,:12].mean(1).sort_values()

6     0.638129
7     0.638835
9     0.640245
5     0.640679
8     0.641113
10    0.642578
12    0.643826
11    0.644314
13    0.644640
14    0.645291
25    0.646322
15    0.647135
24    0.647352
18    0.648383
20    0.648438
16    0.648546
22    0.648655
23    0.648926
17    0.648980
21    0.649523
19    0.649794
dtype: float64

In [327]:
hmod = pd.DataFrame(ntop_list)

hmod.index = hmod.index +1

cols = ["top"+str(c) for c in list(hmod.columns)]

hmod.columns = cols

for c in list(hmod.columns):
    hmod['rank' + (c[3:])] = hmod[c].rank(pct=True)

In [331]:
hmod.iloc[:,:21].mean(1).sort_values()

12    0.643601
2     0.643663
11    0.643756
10    0.643818
9     0.644748
7     0.645213
8     0.645244
3     0.645709
6     0.645740
5     0.646639
4     0.647104
1     0.648593
dtype: float64

In [ ]:
ntop.to_csv('Data/hyperparameters/n_top_experts.csv')
hmod.to_csv('Data/hyperparameters/hist_modifiers.csv')